# Robustness check

In [1]:
import pandas as pd
import numpy as np

from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col # вывод подгонки

import seaborn as sns

# настройки визуализация
import matplotlib.pyplot as plt

# Не показывать Warning
import warnings
warnings.simplefilter(action='ignore', category=Warning)

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<i><b><span style="color: purple">Robustness check</span> </b><br>

Формально __выводы эмпирического анализа зависят от выбранной модели__
(например, значимость нужных коэффициентов).

Может случиться, что при выборе другой модели мы получим противоположные выводы. 


**<span style="color:purple">Возможное решение:</span>** __robustness check__

__Robustness check__:
* Проверяем насколько выводы зависят от выбора модели
* Проверяем устойчивость выводов к изменению базовой модели

**<span style="color:purple">Релизация:</span>** проверяем устойчивость/робастность
* к изменению спецификации (log-переменные, квадраты регрессоров etc)
* к выбору контрольных переменных

</div>

In [2]:
df = pd.read_csv('wage1.csv')
df.shape

(526, 24)

Нас интересует влияние опыта работы `exper` на зарплату `wage`. Рассмотрим несколько моделей с `log(wage)` в качестве зависимой переменной

Обратим внимание на значимость коэффициента при `exper` при разных стандартных уровнях значимости

In [3]:
formulas = ['np.log(wage)~1+exper++female', 
'np.log(wage)~1+exper+female+smsa+married+south+west', 
'np.log(wage)~1+exper+female+I(exper**2)', 
'np.log(wage)~1+exper+female+smsa+married+south+west+I(exper**2)']

mod = []
res = []

for formula in formulas:
	mod.append(ols(formula, df))
	res.append(mod[-1].fit(cov_type='HC3'))

summary_col(res, stars=True)

,np.log(wage) I,np.log(wage) II,np.log(wage) III,np.log(wage) IIII
Intercept,1.7476***,1.4381***,1.5002***,1.2881***
,(0.0398),(0.0605),(0.0483),(0.0615)
exper,0.0038**,0.0022,0.0428***,0.0378***
,(0.0016),(0.0015),(0.0053),(0.0055)
female,-0.3930***,-0.3652***,-0.3796***,-0.3680***
,(0.0425),(0.0405),(0.0406),(0.0390)
smsa,,0.2553***,,0.2454***
,,(0.0449),,(0.0443)
married,,0.2419***,,0.1456***
,,(0.0433),,(0.0440)
